May 17, 2023

# Summarization using Semantic Kernel

This Polyglot notebook C# code written using Visual Studio Code invokes the Semantic Kernel library/SDK Summary skill to summarize a body of text.

GitHub: https://aka.ms/sk/repo

Blog: https://aka.ms/semantic-kernel

Documentation: https://learn.microsoft.com/en-us/semantic-kernel

Post on summarization: https://devblogs.microsoft.com/semantic-kernel/use-the-summarize-skill-to-create-concise-and-actionable-chat-summaries/

## 1. Kernel

### 1.1 Nuget

In [ ]:
//https://www.nuget.org/packages/Microsoft.SemanticKernel/0.14.547.1-preview
#r "nuget: Microsoft.SemanticKernel, 0.14.547.1-preview"

### 1.2 .NET includes

In [ ]:
using System.IO;
using static System.Environment;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;

### 1.3 Kernel instantiation

In [ ]:
IKernel objKernel = Kernel.Builder.Build();

## 2. Planner

### 2.0 Specify endpoint and prompt setting values

In [ ]:
// Azure OpenAI API endpoint
const string OPENAI_URL = "https://[SvcName].openai.azure.com/";
const string OPENAI_MODEL = "[ModelName]";
const string ENV_VAR_NAME_OPENAI_API_KEY = "OpenAIKey";

const EnvironmentVariableTarget ARG_ENV_VAR_TARGET = EnvironmentVariableTarget.User;

string OPENAI_API_KEY = ""; // Initialization with empty string value
OPENAI_API_KEY = GetEnvironmentVariable(ENV_VAR_NAME_OPENAI_API_KEY, ARG_ENV_VAR_TARGET); // Get API key from environment variable

// Prompt configuration settings
var settingMaxTokens = 500; // API call charged for tokens consumed
var settingTemperature = 0.2; // Possible values between 0 and 1; Lower the value, the more the answer is accurate
var settingTopP = 0.5; // Possible values between 0 and 1; Zero is 'Smaller Vocabulary', One is for 'Larger  Vocabulary'

// Source: https://aka.ms/sk/repo/summarize
string strSkillDirectory = "skills"; // Parent folder name
string strSkillName = "SummarizeSkill"; // Folder name
string strFunctionName = "Summarize"; // Sub-folder name

var objSkill = objKernel.ImportSemanticSkillFromDirectory(strSkillDirectory, strSkillName);

### 2.1 Semantic Function or Prompt Template

In [ ]:
// Used in cell 2.2b
string strSemanticFunctionTemplate = """
{{$input}}

Summarize the text content above in under 200 characters.
""";

Console.WriteLine("Semantic Function defined successfully.");

Cells 2.2a and 2.2b are mutually exclusive; only one should occur at a time.

### 2.2a Call semantic function using a prompt-template

In [ ]:
using Microsoft.SemanticKernel.SemanticFunctions;

var promptSettings = new PromptTemplateConfig
{
    Completion =
    {
        MaxTokens = settingMaxTokens, 
        Temperature = settingTemperature, 
        TopP = settingTopP
    }
};

var promptTemplate = new PromptTemplate(
    strSemanticFunctionTemplate, 
    promptSettings, 
    objKernel
);

var functionConfig = new SemanticFunctionConfig(promptSettings, promptTemplate);
var summaryFunction = objKernel.RegisterSemanticFunction(strSkillName, strFunctionName, functionConfig);
Console.WriteLine("Semantic function registered successfully.");

### 2.2b Get reference to the summary function in skill

In [ ]:
using Microsoft.SemanticKernel.Orchestration;
var objSkill = objKernel.ImportSemanticSkillFromDirectory(strSkillDirectory, strSkillName);

string strVarName = "input"; 
// Source: Wikipedia
string strText = "Pakistan is a country in South Asia. It is the world's fifth-most populous country, with a population of almost 243 million people, and has the world's second-largest Muslim population, just behind Indonesia.[16] Islamabad is the nation's capital, while Karachi is its largest city and financial centre, followed by Lahore and Faisalabad. Pakistan is the 33rd-largest country in the world by area and the second-largest in South Asia, spanning 881,913 square kilometres (340,509 square miles). It has a 1,046-kilometre (650-mile) coastline along the Arabian Sea and Gulf of Oman in the south, Afghanistan to the west, Iran to the southwest, and China to the northeast. It is separated narrowly from Tajikistan by Afghanistan's Wakhan Corridor in the north, and also shares a maritime border with Oman.";

var contextVars = new ContextVariables(); 
contextVars.Set(strVarName, strText); 

## 3. Skill

In [ ]:
// Call the Skill function and display the result
var result = await objKernel.RunAsync(contextVars, objSkill[strFunctionName]);
Console.WriteLine(result);

## 4. Memories

In [ ]:
// No Code

## 5. Connectors

In [ ]:
// No Code